# 🎨 NeMo Data Designer: NVINFO Q&A Generator (Seed-Based)

This notebook generates synthetic Q&A datasets based on **real NVINFO content** from the Ground Truth CSV.

## What this notebook does:

1. **Loads real NVINFO Q&A data** from `Ground Truth - NVINFO - Full(Q&A).csv`
2. **Uses it as seed data** to generate similar, high-quality Q&A pairs
3. **Maintains the same categories** (Focal, HR, Benefits, etc.)
4. **Generates realistic answers** with proper context from NVINFO

---

## 📦 Import Dependencies

In [35]:
from data_designer.essentials import (
    DataDesignerConfigBuilder,
    ChatCompletionInferenceParams,
    LLMJudgeColumnConfig,
    LLMTextColumnConfig,
    ModelConfig,
    DataDesigner,
    Score,
    LocalFileSeedSource,
)
import pandas as pd

## 📊 Load Ground Truth Data

In [36]:
# Load the Ground Truth NVINFO Q&A dataset
ground_truth_df = pd.read_csv('Ground_Truth_NVINFO_QA_UTF8.csv')

print(f"Loaded {len(ground_truth_df)} Q&A pairs from Ground Truth")
print(f"\nCategories: {sorted(ground_truth_df['Category'].unique().tolist())}")
print(f"\nSample data:")
ground_truth_df[['Category', 'Query', 'Correct Answer']].head(3)

Loaded 1303 Q&A pairs from Ground Truth

Categories: ['Benefits', 'Brand', 'Clubs', 'Collaboration', 'Company Strategy', 'Contracts', 'Data Privacy', 'Discounts & Services', 'Diversity, Inclusion, and Belonging', 'Employee Listening', 'Environmental Health & Safety', 'Finance, Stock', 'Focal', 'Form Center', 'GTC', 'Global Security', 'Green 2 Work', 'HR', 'Information Security', 'Inside NVIDIA', 'Inspire 365', 'Learning & Development', 'Legal, Shipping', 'My Career', 'NTECH', 'NVHelp', 'NVHelp, Corporate Calendar', 'NVIDIA Developer Blogs', 'NVIDIA Travel', 'NVINFO Home Page', 'NVINFO-China', 'NVINFO-India', 'NVINFO-Israel', 'NVINFO-US', 'NVINFO-Updates', 'Nursing Rooms', 'Patents, Legal', 'Payroll', 'Policies', 'Recognition', 'Referrals', 'Retirement', 'SC Cafes', 'Scout', 'Site Services', 'Speak Up', 'Staffing', 'Tech Docs', 'Trademarks, Marketing', 'Trustworthy AI', 'U.S. Benefits News & Info', 'Well-Being', 'nvidia.com']

Sample data:


,Category,Query,Correct Answer
0,Focal,How can I access this year's Focal pay statement?,Employees can access pay statement in Workday....
1,Focal,How can I get a copy of last year's pay statem...,Employees can access their pay statement in Wo...
2,Focal,Why am I seeing two grants during this Focal c...,Employees who received a grant as part of this...


## ⚙️ Initialize Data Designer

In [37]:
data_designer = DataDesigner()

In [38]:

MODEL_PROVIDER = "nvcf"  
JUDGE_MODEL_PROVIDER = "nvcf"

MODEL_ID = "nvidia/nvidia/Nemotron-3-Nano-30B-A3B"
JUDGE_MODEL_ID = "nvidia/openai/gpt-oss-120b"

MODEL_ALIAS = "nemotron-3-nano"
JUDGE_MODEL_ALIAS = "gpt-oss-120b"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=ChatCompletionInferenceParams(
            temperature=1.0,
            top_p=1.0,
            max_tokens=1024,
        ),
    ),

    ModelConfig(
        alias=JUDGE_MODEL_ALIAS,
        model=JUDGE_MODEL_ID,
        provider=JUDGE_MODEL_PROVIDER,
        inference_parameters=ChatCompletionInferenceParams(
            temperature=0.6,
            top_p=0.95,
            max_tokens=1024,
        ),
    )
]

## 🏗️ Build Configuration with Seed Data

In [39]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 📚 Configure Seed Dataset

In [40]:
# Use the UTF-8 cleaned Ground Truth CSV as seed data
seed_source = LocalFileSeedSource(path="Ground_Truth_NVINFO_QA_UTF8.csv")
config_builder.with_seed_dataset(seed_source)
print("✓ Seed dataset configured")

✓ Seed dataset configured


## 🦜 Generate New Questions Based on Seed Context

In [41]:
# Generate a NEW question similar to the seed question
config_builder.add_column(
    LLMTextColumnConfig(
        name="new_question",
        model_alias=MODEL_ALIAS,
        prompt=(
            "You are an NVIDIA employee asking questions about internal resources and policies.\n\n"
            "Here is an example question from the {{ Category }} category:\n"
            "{{ Query }}\n\n"
            "Generate a NEW, DIFFERENT question that is similar in style and topic, "
            "but asks about a different aspect of {{ Category }}.\n"
            "Keep the question natural, concise, and employee-focused.\n\n"
            "IMPORTANT: Output plain text only. Do NOT use markdown formatting, asterisks, bullets, or numbered lists.\n"
            "Use ONLY standard ASCII characters. Use regular hyphens (-), not en-dashes or em-dashes. "
            "Use straight quotes (\"), not curly quotes. Do NOT use accented characters."
        ),
    )
)
print("✓ Added new_question column")

✓ Added new_question column


## 💬 Generate Answers Based on NVINFO Context

In [42]:
# Generate an answer using the original answer as context/reference
config_builder.add_column(
    LLMTextColumnConfig(
        name="new_answer",
        model_alias=MODEL_ALIAS,
        prompt=(
            "You are an NVINFO assistant. Answer this question about {{ Category }}: {{ new_question }}\n\n"
            "Reference information from NVINFO:\n"
            "{{ ['Correct Answer'] }}\n\n"
            "Provide a clear, direct answer based on the reference information. "
            "DO NOT repeat or rephrase the question. "
            "Start your response with the answer immediately. "
            "Keep it professional, accurate, and concise. "
            "Provide a SHORT, direct summary. Keep only the essential information. Remove any redundant details. Keep it under 100 words.\n\n"
            "IMPORTANT: Use ONLY standard ASCII characters. Use regular hyphens (-), not en-dashes or em-dashes. "
            "Use straight quotes (\"), not curly quotes. Do NOT use markdown formatting like bold (**text**) or arrows (→). "
            "Do NOT use accented characters (use 'resume' not 'résumé')."
        ),
    )
)
print("✓ Added new_answer column")

✓ Added new_answer column


## 🔍 Quality Assessment with LLM-as-a-Judge

In [43]:
# Define evaluation rubrics
RelevanceRubric = Score(
    name="Relevance",
    description="How relevant is the generated answer to the question and category?",
    options={
        "HighlyRelevant": "The answer directly addresses the question and is appropriate for the category.",
        "SomewhatRelevant": "The answer is related but may miss some key points or be partially off-topic.",
        "NotRelevant": "The answer does not adequately address the question or is off-topic.",
    },
)

AccuracyRubric = Score(
    name="Accuracy",
    description="How accurate is the information compared to the reference NVINFO content?",
    options={
        "Accurate": "The information aligns well with the reference content and NVIDIA policies.",
        "PartiallyAccurate": "Some information is correct but there are minor inaccuracies or omissions.",
        "Inaccurate": "The information contradicts the reference content or contains significant errors.",
    },
)

# Evaluate quality
config_builder.add_column(
    LLMJudgeColumnConfig(
        name="quality_metrics",
        model_alias=JUDGE_MODEL_ALIAS,
        prompt=(
            "Category: {{ Category }}\n"
            "Question: {{ new_question }}\n"
            "Generated Answer: {{ new_answer }}\n\n"
            "Reference Information from NVINFO:\n{{ ['Correct Answer'] }}\n\n"
            "Evaluate the quality of the generated answer."
        ),
        scores=[RelevanceRubric, AccuracyRubric],
    )
)
print("✓ Added quality assessment")

✓ Added quality assessment


## 🔁 Preview the Generated Data

In [44]:
# Preview a few records (this will take 1-2 minutes)
print("🚀 Generating preview with 3 Q&A pairs...")
print("⏳ Please wait...\n")

preview = data_designer.preview(config_builder, num_records=3)

[23:38:03] [INFO] 📺 Preview generation in progress
[23:38:03] [INFO] ✅ Validation passed
[23:38:03] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[23:38:03] [INFO] 🩺 Running health checks for models...
[23:38:03] [INFO]   |-- 👀 Checking 'nvidia/nvidia/Nemotron-3-Nano-30B-A3B' in provider named 'nvcf' for model alias 'nemotron-3-nano'...


🚀 Generating preview with 3 Q&A pairs...
⏳ Please wait...



[23:38:04] [INFO]   |-- ✅ Passed!
[23:38:04] [INFO]   |-- 👀 Checking 'nvidia/openai/gpt-oss-120b' in provider named 'nvcf' for model alias 'gpt-oss-120b'...
[23:38:05] [INFO]   |-- ✅ Passed!
[23:38:05] [INFO] 🌱 Sampling 3 records from seed dataset
[23:38:05] [INFO]   |-- seed dataset size: 1303 records
[23:38:05] [INFO]   |-- sampling strategy: ordered
[23:38:05] [INFO] 📝 llm-text model config for column 'new_question'
[23:38:05] [INFO]   |-- model: 'nvidia/nvidia/Nemotron-3-Nano-30B-A3B'
[23:38:05] [INFO]   |-- model alias: 'nemotron-3-nano'
[23:38:05] [INFO]   |-- model provider: 'nvcf'
[23:38:05] [INFO]   |-- inference parameters: generation_type=chat-completion, max_parallel_requests=4, temperature=1.00, top_p=1.00, max_tokens=1024
[23:38:05] [INFO] 🐙 Processing llm-text column 'new_question' with 4 concurrent workers
[23:38:08] [INFO] 📝 llm-text model config for column 'new_answer'
[23:38:08] [INFO]   |-- model: 'nvidia/nvidia/Nemotron-3-Nano-30B-A3B'
[23:38:08] [INFO]   |-- model

In [45]:
# Display sample records
preview.display_sample_record()

                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name              ┃ Value                                                                                       ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ID                │ 1                                                                                           │
├───────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ Query             │ How can I access this year's Focal pay statement?                                           │
├───────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ Category          │ Focal                                                                                       │
├───────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ Correct Answer    │ Employees can access pay statement in Workday. Once statements are available for this       │
│                   │ cycle, follow this [step-by-step                                                            │
│                   │ guide](https://nvidia.sharepoint.com/%3ab%3a/r/sites/nvinfo/focal/Documents/WD-Comp-Statem… │
├───────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ Required Citation │ https://nvidia.sharepoint.com/sites/nvinfo/focal/Pages/focal-faq.aspx                       │
├───────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ Location          │                                                                                             │
└───────────────────┴─────────────────────────────────────────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name         ┃ Value                                                                                            ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ new_question │ Where can I retrieve my latest Focal compensation document?                                      │
├──────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────┤
│ new_answer   │ Your latest Focal compensation document is stored in the NVINFO portal under Compensation >      │
│              │ Focal > Documents, where you can download it.                                                    │
└──────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                          LLM-as-a-Judge: quality_metrics                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Relevance                                              ┃ Accuracy                                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [46]:
# View the generated dataset
df_preview = preview.dataset  # For preview: use .dataset (not .load_dataset())
df_preview[['Category', 'Query', 'new_question', 'new_answer']]

,Category,Query,new_question,new_answer
0,Focal,How can I access this year's Focal pay statement?,Where can I retrieve my latest Focal compensat...,Your latest Focal compensation document is sto...
1,Focal,How can I get a copy of last year's pay statem...,How do I access the updated benefits enrollmen...,Updated benefits enrollment guide can be acces...
2,Focal,Why am I seeing two grants during this Focal c...,What triggers a duplicate grant entry in the c...,Duplicate grant entry is triggered when a gran...


## 🆙 Scale Up: Generate Full Dataset

In [47]:
# Generate a larger dataset (e.g., 100 Q&A pairs)
print("🚀 Generating 100 Q&A pairs...")
print("⏳ This will take several minutes...\n")

job_results = data_designer.create(config_builder, num_records=100)

[23:38:23] [INFO] 🎨 Creating Data Designer dataset
[23:38:23] [INFO] ✅ Validation passed
[23:38:23] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[23:38:23] [INFO] 🩺 Running health checks for models...
[23:38:23] [INFO]   |-- 👀 Checking 'nvidia/nvidia/Nemotron-3-Nano-30B-A3B' in provider named 'nvcf' for model alias 'nemotron-3-nano'...


🚀 Generating 100 Q&A pairs...
⏳ This will take several minutes...



[23:38:24] [INFO]   |-- ✅ Passed!
[23:38:24] [INFO]   |-- 👀 Checking 'nvidia/openai/gpt-oss-120b' in provider named 'nvcf' for model alias 'gpt-oss-120b'...
[23:38:25] [INFO]   |-- ✅ Passed!
[23:38:25] [INFO] 📂 Dataset path '/mnt/c/Users/maggiez/Documents/datadesigner/DataDesigner/artifacts/dataset' already exists. Dataset from this session
		     will be saved to '/mnt/c/Users/maggiez/Documents/datadesigner/DataDesigner/artifacts/dataset_01-31-2026_233825' instead.
[23:38:25] [INFO] ⏳ Processing batch 1 of 1
[23:38:25] [INFO] 🌱 Sampling 100 records from seed dataset
[23:38:25] [INFO]   |-- seed dataset size: 1303 records
[23:38:25] [INFO]   |-- sampling strategy: ordered
[23:38:25] [INFO] 📝 llm-text model config for column 'new_question'
[23:38:25] [INFO]   |-- model: 'nvidia/nvidia/Nemotron-3-Nano-30B-A3B'
[23:38:25] [INFO]   |-- model alias: 'nemotron-3-nano'
[23:38:25] [INFO]   |-- model provider: 'nvcf'
[23:38:25] [INFO]   |-- inference parameters: generation_type=chat-completion,

In [48]:
# Load and inspect the generated dataset
dataset = job_results.load_dataset()
print(f"✅ Generated {len(dataset)} Q&A pairs!\n")
dataset.head(10)

✅ Generated 100 Q&A pairs!



,ID,Query,Category,Correct Answer,Required Citation,Location,new_question,new_question__reasoning_trace,new_answer,new_answer__reasoning_trace,quality_metrics,quality_metrics__reasoning_trace
0,1,How can I access this year's Focal pay statement?,Focal,Employees can access pay statement in Workday....,https://nvidia.sharepoint.com/sites/nvinfo/foc...,None,Where can I access the most recent Focal proje...,"We need to output a question, plain text, no m...",The most recent Focal project guidelines are a...,We need to answer as an NVINFO assistant. The ...,{'Accuracy': {'reasoning': 'The provided locat...,We need to evaluate the generated answer. The ...
1,2,How can I get a copy of last year's pay statem...,Focal,Employees can access their pay statement in Wo...,https://nvidia.sharepoint.com/sites/nvinfo/foc...,None,Who should I contact to resolve issues with my...,"We need to output a question, plain text, no f...",Contact your Knowledge Base Administrator.,We need to answer as an NVINFO assistant. Prov...,{'Accuracy': {'reasoning': 'The provided answe...,"We need to evaluate the generated answer: ""Con..."
2,3,Why am I seeing two grants during this Focal c...,Focal,Employees who received a grant as part of this...,https://nvidia.sharepoint.com/sites/nvinfo/foc...,None,How do I adjust my Focal priorities?,"We need to produce a plain text question, simi...","In Focal, open the Priorities panel, modify ea...","We need to answer: ""How do I adjust my Focal p...",{'Accuracy': {'reasoning': 'The provided steps...,We need to evaluate generated answer relative ...
3,4,"On my Focal pay statement, what does Total Ann...",Focal,Total Annual Pay represents your total cash co...,https://nvidia.sharepoint.com/sites/nvinfo/foc...,None,How does the Focal equity grant vesting schedu...,We need to produce a question similar style an...,The vesting schedule ties equity awards to tim...,"We need to answer: ""How does the Focal equity ...",{'Accuracy': {'reasoning': 'The explanation al...,We need to evaluate the generated answer vs re...
4,5,What is the vesting schedule for this year's F...,Focal,Grants will vest according to a four-year sche...,https://nvidia.sharepoint.com/sites/nvinfo/foc...,None,Are there any restrictions on combining multip...,The user wants an employee asking a question a...,No – there are no restrictions on combining mu...,"We need to answer: ""Are there any restrictions...",{'Accuracy': {'reasoning': 'The answer matches...,"We need to evaluate the generated answer: ""No ..."
5,6,What is the objective of the Performance Manag...,Focal,"Broadly, the Performance Management process en...",https://nvidia.sharepoint.com/sites/nvinfo/foc...,None,What is the process for requesting a flexible ...,"We need to produce a plain text question, simi...","Submit a written request to your manager, comp...",We need to answer about process for requesting...,{'Accuracy': {'reasoning': 'Based on typical N...,We need to evaluate generated answer vs refere...
6,7,What's typically included in the Performance M...,Focal,NVIDIAs Performance Management starts with al...,https://nvidia.sharepoint.com/sites/nvinfo/foc...,None,How do I set and update my objectives in Focal?,We need to produce a new question similar styl...,"Navigate to the Objectives tab, click Set Obje...","The user wants answer about ""Focal: How do I s...",{'Accuracy': {'reasoning': 'The answer provide...,We need to evaluate generated answer vs refere...
7,8,What happens in a Focal process?,Focal,"In the Focal process, managers: 1. Assess emp...",https://nvidia.sharepoint.com/sites/nvinfo/foc...,None,Who is responsible for setting the objectives ...,"We need to produce a plain text question, simi...",The manager is responsible for setting the obj...,"We need to answer: ""Who is responsible for set...",{'Accuracy': {'reasoning': 'The answer matches...,"We need to evaluate the generated answer: ""The..."
8,9,What is the eligibility criteria for cash and ...,Focal,"Regular, full-time em

In [49]:
# Analyze quality metrics
analysis = job_results.load_analysis()
analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 100                             │ 3                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                   ┃               ┃                              ┃      prompt tokens ┃       completion tokens ┃
┃ column name       ┃     data type ┃         number unique values ┃         per record ┃              per record ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ new_question      │        string │                   99 (99.0%) │      134.0 +/- 5.7 │            13.0 +/- 4.2 │
├───────────────────┼───────────────┼──────────────────────────────┼────────────────────┼─────────────────────────┤
│ new_answer        │        string │                   86 (86.0%) │      166.0 +/- 4.5 │           18.5 +/- 12.1 │
└───────────────────┴───────────────┴──────────────────────────────┴────────────────────┴─────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                ⚖️ LLM-Judge Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃                      ┃               ┃                            ┃       prompt tokens ┃     completion tokens ┃
┃ column name          ┃     data type ┃       number unique values ┃          per record ┃            per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ quality_metrics      │          dict │               100 (100.0%) │      591.0 +/- 13.0 │         89.5 +/- 14.8 │
└──────────────────────┴───────────────┴────────────────────────────┴─────────────────────┴───────────────────────┘
                                                                                                                   
                                                                                                                   
╭────────────────────────────────────────────────── Table Notes ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  1. All token statistics are based on a sample of max(1000, len(dataset)) records.                              │
│  2. Tokens are calculated using tiktoken's cl100k_base tokenizer.                                               │
│                                                       

## 💾 Save the Generated Dataset

In [51]:
# Save TWO output files
import json

print("💾 Saving generated data...\n")

# Create a copy for processing
df_save = dataset.copy()

# Extract LLM-as-a-Judge scores into separate columns
if 'quality_metrics' in df_save.columns:
    df_save['Relevance_Score'] = df_save['quality_metrics'].apply(
        lambda x: x.get('Relevance', {}).get('score') if isinstance(x, dict) else None
    )
    df_save['Accuracy_Score'] = df_save['quality_metrics'].apply(
        lambda x: x.get('Accuracy', {}).get('score') if isinstance(x, dict) else None
    )

# ─────────────────────────────────────────────────────────────────
# FILE 1: Simple Q&A Only (just generated queries and answers)
# ─────────────────────────────────────────────────────────────────
qa_only_file = 'nvinfo_generated_QA_only.csv'
df_qa_only = df_save[['new_question', 'new_answer']].copy()
df_qa_only.to_csv(qa_only_file, index=False)

print("📄 File 1: Q&A Only")
print(f"   Filename: {qa_only_file}")
print(f"   Columns: new_question, new_answer")
print(f"   Records: {len(df_qa_only)}")
print(f"   ✅ Saved!\n")

# ─────────────────────────────────────────────────────────────────
# FILE 2: Complete Dataset (everything including LLM-as-a-Judge)
# ─────────────────────────────────────────────────────────────────
complete_file = 'nvinfo_generated_COMPLETE.csv'
complete_columns = [
    # Original seed data
    'ID',
    'Category',
    'Query',
    'Correct Answer',
    'Required Citation',
    'Location',
    # Generated columns
    'new_question',
    'new_answer',
    # LLM-as-a-Judge scores
    'Relevance_Score',
    'Accuracy_Score',
]

# Only include columns that exist
existing_columns = [col for col in complete_columns if col in df_save.columns]
df_save[existing_columns].to_csv(complete_file, index=False)

print("📋 File 2: Complete Dataset")
print(f"   Filename: {complete_file}")
print(f"   Includes:")
print(f"     - Original seed data (Category, Query, Correct Answer, etc.)")
print(f"     - Generated columns (new_question, new_answer)")
print(f"     - LLM-as-a-Judge scores (Relevance_Score, Accuracy_Score)")
print(f"   Records: {len(df_save)}")
print(f"   ✅ Saved!\n")

print("="*60)
print("✅ Both files saved successfully!")
print("="*60)

💾 Saving generated data...

📄 File 1: Q&A Only
   Filename: nvinfo_generated_QA_only.csv
   Columns: new_question, new_answer
   Records: 100
   ✅ Saved!

📋 File 2: Complete Dataset
   Filename: nvinfo_generated_COMPLETE.csv
   Includes:
     - Original seed data (Category, Query, Correct Answer, etc.)
     - Generated columns (new_question, new_answer)
     - LLM-as-a-Judge scores (Relevance_Score, Accuracy_Score)
   Records: 100
   ✅ Saved!

✅ Both files saved successfully!


## 🎯 Filter High-Quality Data (Accurate Only)

Remove low-quality entries to create a production-ready dataset.

In [52]:
# ─────────────────────────────────────────────────────────────────
# FILE 3: Accurate Dataset (only high-quality Q&A)
# ─────────────────────────────────────────────────────────────────
print("\n📊 Filtering high-quality data...\n")

# Define columns to save
complete_columns = [
    'ID', 'Category', 'Query', 'Correct Answer',
    'Relevance_Score', 'Accuracy_Score',
]
existing_columns = [col for col in complete_columns if col in df_save.columns]
columns_to_save = [col for col in existing_columns if col != 'quality_metrics']

# Filter based on score labels
# Keep only: HighlyRelevant AND Accurate
df_accurate = df_save[
    (df_save['Relevance_Score'] == 'HighlyRelevant') & 
    (df_save['Accuracy_Score'] == 'Accurate')
].copy()

# Save the filtered dataset
accurate_file = 'nvinfo_generated_accurate.csv'
df_accurate[columns_to_save].to_csv(accurate_file, index=False, encoding='utf-8')

print("✨ File 3: High-Quality (Accurate) Dataset")
print(f"   Filename: {accurate_file}")
print(f"   Filters applied:")
print(f"     - Relevance_Score = 'HighlyRelevant'")
print(f"     - Accuracy_Score = 'Accurate'")
print(f"   (Removes: NotRelevant, SomewhatRelevant, Inaccurate, PartiallyAccurate)")
print(f"   Original records: {len(df_save)}")
print(f"   Filtered records: {len(df_accurate)}")
print(f"   Removed: {len(df_save) - len(df_accurate)} low-quality rows")
print(f"   Quality rate: {len(df_accurate)/len(df_save)*100:.1f}%")
print(f"   ✅ Saved!\n")

print("="*60)
print("📊 Summary of All Generated Files:")
print("="*60)
print(f"1. {qa_only_file} - Simple Q&A ({len(df_qa_only)} records)")
print(f"2. {complete_file} - Complete dataset ({len(df_save)} records)")
print(f"3. {accurate_file} - High-quality only ({len(df_accurate)} records)")
print("="*60)


📊 Filtering high-quality data...

✨ File 3: High-Quality (Accurate) Dataset
   Filename: nvinfo_generated_accurate.csv
   Filters applied:
     - Relevance_Score = 'HighlyRelevant'
     - Accuracy_Score = 'Accurate'
   (Removes: NotRelevant, SomewhatRelevant, Inaccurate, PartiallyAccurate)
   Original records: 100
   Filtered records: 78
   Removed: 22 low-quality rows
   Quality rate: 78.0%
   ✅ Saved!

📊 Summary of All Generated Files:
1. nvinfo_generated_QA_only.csv - Simple Q&A (100 records)
2. nvinfo_generated_COMPLETE.csv - Complete dataset (100 records)
3. nvinfo_generated_accurate.csv - High-quality only (78 records)
